This Jupyter Notebook can be used to reproduce the results in all figures in our paper "Domain Adaptation for Real-World Single View 3D Reconstruction" (Brandon Leung, Siddharth Singh, Arik Horodniceanu) for CSE 291: Domain Adaptation in Computer Vision.

First, please download our source code here:


Python 3.7 is recommended. Note that the following depencencies are required:
* argparse
* easydict
* matplotlib
* numpy
* opencv-python
* pprint
* scipy
* pytorch
* torchvision
* pandas
* tqdm
* sklearn

You will also need to download the following datasets:
* Shapenet rendering images http://cvgl.stanford.edu/data2/ShapeNetRendering.tgz
* shapenet voxelized models http://cvgl.stanford.edu/data2/ShapeNetVox32.tgz
* ODDS dataset https://drive.google.com/open?id=1ZrljEGjqjduODVEXYL4dvIVO-DQzakc2

Once downloaded, please update config.py with the location of the downloaded datasets on your computer.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# A GPU is required. If there are multiple GPUs, please set the desired one here.
from config import cfg
import os
cfg.CONST.DEVICE = "3"
# need to set gpus before anything involving pytorch
os.environ["CUDA_VISIBLE_DEVICES"] = cfg.CONST.DEVICE

In [3]:
from core.train import train_net
from core.test import test_net
import torch
import pprint
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np

# t-SNE Results

In [4]:
def embed_TSNE(data):
    X_embedded = data
    X_embedded = PCA(n_components = 50, svd_solver='auto').fit_transform(X_embedded)
    X_embedded = TSNE(n_components=2).fit_transform(X_embedded) 
    return X_embedded

def visualize_tsne(embedded_latent_vectors, class_labels, title="TSNE"):
    
    embedded_df = pd.DataFrame(embedded_latent_vectors) 
    ax = embedded_df.plot.scatter(x=0, y = 1, c = class_labels, colormap="viridis")
    ax.set_title(title)

In [ ]:
trial_names = []
weights_paths = [
    "./output_submission/author_saved_params/Pix2Vox-A-ShapeNet.pth",
    "./output_submission/2020_03_19--18_54_21_OOWL_DA_CORAL_lam_10_TRAIN/checkpoints/best-ckpt.pth",
    "./output_submission/2020_03_19--21_49_02_OOWL_DA_DANN_lam_1_TRAIN/checkpoints/best-ckpt.pth",
]

for weights_path in weights_paths:
    cfg.CONST.WEIGHTS = weights_path

In [6]:
cfg.CONST.WEIGHTS = "./output_submission/2020_03_21--01_49_04_OWILDVoxelCls_VoxLamExp_DANNlam1_TRAIN/checkpoints/best-ckpt.pth"

# obtaining latent vectors for target domain
cfg.DATASET.CLASSES_TO_USE = ["aeroplane", "car", "display", "lamp", "telephone", "watercraft"]
cfg.DATASET.TEST_DATASET = "OWILD"
cfg.TEST.USE_TRAIN_SET = True
cfg.CONST.N_VIEWS_RENDERING = 8
_, t_class_labels, t_latent_vectors = test_net(cfg)
t_embedded = embed_TSNE(t_latent_vectors)

# obtaining latent vectors for ShapeNet source domain
cfg.DATASET.CLASSES_TO_USE = ["aeroplane"]
#cfg.DATASET.CLASSES_TO_USE = ["aeroplane", "car", "display", "lamp", "telephone", "watercraft"]
cfg.DATASET.TEST_DATASET = "ShapeNet"
cfg.CONST.N_VIEWS_RENDERING=1

_, s_class_labels, s_latent_vectors = test_net(cfg)
s_embedded = embed_TSNE(s_latent_vectors)

# show TSNE for source and target domains
s_t_embedded = np.concatenate((s_embedded, t_embedded), axis =0)
domain_labels = [0 for i in range(s_embedded.shape[0])] + [1 for i in range(t_embedded.shape[0])]
visualize_tsne(s_t_embedded, domain_labels, "Source and Target Domain t-SNE")

[INFO] 2020-03-21 16:27:27.645588 Collecting files of Taxonomy[ID=Airplane_Model, Name=Airplane_Model]
[INFO] 2020-03-21 16:27:27.647326 Collecting files of Taxonomy[ID=Boat_Model, Name=Boat_Model]
[INFO] 2020-03-21 16:27:27.648714 Collecting files of Taxonomy[ID=Car_Model, Name=Car_Model]
[INFO] 2020-03-21 16:27:27.649912 Collecting files of Taxonomy[ID=Lamp, Name=Lamp]
[INFO] 2020-03-21 16:27:27.651050 Collecting files of Taxonomy[ID=Monitor, Name=Monitor]
[INFO] 2020-03-21 16:27:27.652386 Collecting files of Taxonomy[ID=Telephone, Name=Telephone]
[INFO] 2020-03-21 16:27:27.653757 Complete collecting files of the dataset. Total files: 93.
[INFO] 2020-03-21 16:27:35.411815 Loading weights from ./output_submission/2020_03_21--01_49_04_OWILDVoxelCls_VoxLamExp_DANNlam1_TRAIN/checkpoints/best-ckpt.pth ...


FileNotFoundError: [Errno 2] No such file or directory: './output_submission/2020_03_21--01_49_04_OWILDVoxelCls_VoxLamExp_DANNlam1_TRAIN/checkpoints/best-ckpt.pth'